# Gallery Example: M/M/1-PS Reentrant Queue

This example demonstrates an M/M/1 queueing system with Processor Sharing and reentrant routing:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Exponential
- **Servers**: 1 server
- **Scheduling**: Processor Sharing (PS)
- **Reentrant**: Jobs switch class and pass through the queue again

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mm1_ps_reentrant():
    """Create M/M/1-PS-Reentrant queueing model"""
    model = Network('M/M/1')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.PS)  # Processor Sharing
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    oclass2 = OpenClass(model, 'Class2')
    
    # Exponential arrivals for Class1, Class2 is disabled at source
    source.set_arrival(oclass1, Exp(1))
    source.set_arrival(oclass2, Disabled())
    
    # Exponential service for both classes
    queue.set_service(oclass1, Exp(2))
    queue.set_service(oclass2, Exp(3))
    
    # Block 3: topology with class switching (reentrant)
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_class_switch(oclass1, oclass2, queue, queue, 1.0)
    P.add_route(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model

# Create the model
model = gallery_mm1_ps_reentrant()

## About Processor Sharing with Reentrant Routing

**Processor Sharing (PS)** scheduling:
- All jobs share the server capacity equally
- Each of n jobs receives 1/n of the server capacity
- Fair scheduling independent of service time

**With reentrant routing**:
- Jobs pass through the queue twice (as Class1, then Class2)
- Under PS, both classes share capacity simultaneously
- The mean response time depends on total work in the system

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)